In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import warnings
warnings.filterwarnings("ignore")

In [37]:
def interpolate(df):
    df = df.copy()
    df['ts'] = pd.to_datetime(df['timestamp'])
    df = df.set_index('ts')
    df = df.interpolate(method='time', limit_direction='both')    
    return df

In [2]:
DATA_DIR = 'data/'
SUBSAMPLE_FEATURE_PREPROC = 0.01
SUBSAMPLE_TRAIN = 0.5

In [3]:
!ls {DATA_DIR}

building_metadata.csv  test.csv   weather_test.csv
sample_submission.csv  train.csv  weather_train.csv


# Load data 

In [4]:
building_metadata = pd.read_csv(DATA_DIR + 'building_metadata.csv')
weather_train = pd.read_csv(DATA_DIR + 'weather_train.csv')
weather_test = pd.read_csv(DATA_DIR + 'weather_test.csv')
train = pd.read_csv(DATA_DIR + 'train.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')

In [5]:
map_meter2desc = {0: 'electricity', 1: 'chilledwater', 2: 'steam', 3: 'hotwater'}

In [6]:
numerical = [
    'square_feet', 
    'year_built', 
    'floor_count', 
    'air_temperature', 
    'cloud_coverage', 
    'dew_temperature',
    'precip_depth_1_hr',
    'sea_level_pressure',
    'wind_speed',
]
numerical_aug = [
    'wind_direction_sin',
    'wind_direction_cos',
]

categorical = [
    'building_id',
    'meter',
    'primary_use',
]
categorical_aug = [
    'month',
]

feature_names = numerical + categorical + numerical_aug + categorical_aug

# Combine table

In [32]:
df = weather_train.copy()


In [ ]:
# Interpolate the weather data
def interpolate(df):
    

In [22]:
train.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [39]:
df = train
df = df.merge(building_metadata, on='building_id', how='left')

weather_train_full = df[['site_id', 'timestamp']].drop_duplicates()\
    .merge(weather_train, on=['site_id', 'timestamp'], how='left')\
    .groupby(by='site_id', group_keys=False).apply(interpolate).reset_index(drop=True)
    
df = df.merge(weather_train_full, on=['site_id', 'timestamp'], how='left')

In [42]:
df['log_meter_reading'] = df['meter_reading'].apply(lambda x: np.log(x + 1))
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Remove the problematic data

In [43]:
df = df.query('not (meter == 0 and site_id == 0 and timestamp < "2016-05-21")').reset_index(drop=True)

# Feature pre-processing

In [44]:
df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,log_meter_reading
0,105,0,2016-01-01,23.3036,1,Education,50623,NaN,5.0,3.8,0.0,2.4,NaN,1020.9,240.0,3.1,3.190624
1,106,0,2016-01-01,0.3746,1,Education,5374,NaN,4.0,3.8,0.0,2.4,NaN,1020.9,240.0,3.1,0.318163
2,106,3,2016-01-01,0.0000,1,Education,5374,NaN,4.0,3.8,0.0,2.4,NaN,1020.9,240.0,3.1,0.000000
3,107,0,2016-01-01,175.1840,1,Education,97532,2005.0,10.0,3.8,0.0,2.4,NaN,1020.9,240.0,3.1,5.171529
4,108,0,2016-01-01,91.2653,1,Education,81580,1913.0,5.0,3.8,0.0,2.4,NaN,1020.9,240.0,3.1,4.524668


In [45]:
from sklearn.pipeline import FeatureUnion, make_pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.compose import make_column_transformer

feature_preproc = make_pipeline(
    FeatureUnion([
        ('numeric_features', make_pipeline(
            FunctionTransformer(lambda x: x[numerical], validate=False),
            FunctionTransformer(lambda x: x.assign(**{'precip_depth_1_hr': lambda y: y['precip_depth_1_hr'].clip(lower=0).fillna(0)}), validate=False),
            SimpleImputer(strategy="median"),
            StandardScaler(),
        )),
        ('categorical_features', make_pipeline(
            FunctionTransformer(lambda x: x[categorical], validate=False),
            SimpleImputer(strategy="most_frequent"),
            OrdinalEncoder(),
        )),
        ('wind_direction', make_pipeline(
            FunctionTransformer(lambda x: pd.concat((
                    x['wind_direction'].apply(lambda x: np.sin(x/360 * 2*np.pi)),
                    x['wind_direction'].apply(lambda x: np.cos(x/360 * 2*np.pi)),
                ), axis='columns'), validate=False),
            SimpleImputer(strategy="median"),
        )),
        ('month', make_pipeline(
            FunctionTransformer(lambda x: x['timestamp'].dt.month.to_frame(), validate=False),
            SimpleImputer(strategy="most_frequent"),
            OrdinalEncoder(),
        )), 
     ]),
    FunctionTransformer(
        lambda x: pd.DataFrame(x, columns=feature_names).astype(float), 
        validate=False,
    ),
)

feature_preproc.fit(df.sample(int(SUBSAMPLE_FEATURE_PREPROC * df.shape[0]), random_state=42));



In [56]:
# np.random.seed(42)
# idx_train = np.random.choice(df.shape[0], int(SUBSAMPLE_TRAIN * df.shape[0]), replace=False)

# idx = np.zeros(df.shape[0]).astype(bool)
# idx[idx_train] = True
# idx_train = idx

In [46]:
np.random.seed(42)
timestamp_train = np.random.choice(df['timestamp'].unique(), int(SUBSAMPLE_TRAIN * df['timestamp'].nunique()), replace=False)

idx_train = df['timestamp'].isin(timestamp_train).values

In [47]:
X_train = feature_preproc.transform(df.iloc[idx_train, :])
y_train = df.iloc[idx_train]['log_meter_reading']

In [48]:
X_val = feature_preproc.transform(df.iloc[~idx_train, :])
y_val = df.iloc[~idx_train]['log_meter_reading']

In [49]:
X_train.head()

,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_speed,building_id,meter,primary_use,wind_direction_sin,wind_direction_cos,month
0,-0.490197,0.020934,1.022735,-1.097114,-0.994408,-0.513206,-0.157098,0.703926,-0.119416,105.0,0.0,0.0,-0.866025,-0.5,0.0
1,-0.874430,0.020934,0.451809,-1.097114,-0.994408,-0.513206,-0.157098,0.703926,-0.119416,106.0,0.0,0.0,-0.866025,-0.5,0.0
2,-0.874430,0.020934,0.451809,-1.097114,-0.994408,-0.513206,-0.157098,0.703926,-0.119416,106.0,3.0,0.0,-0.866025,-0.5,0.0
3,-0.091868,1.982767,3.877363,-1.097114,-0.994408,-0.513206,-0.157098,0.703926,-0.119416,107.0,0.0,0.0,-0.866025,-0.5,0.0
4,-0.227325,-2.895305,1.022735,-1.097114,-0.994408,-0.513206,-0.157098,0.703926,-0.119416,108.0,0.0,0.0,-0.866025,-0.5,0.0


# Train

In [53]:
import lightgbm as lgb

d_train = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical + categorical_aug)
d_val = lgb.Dataset(X_val, label=y_val, categorical_feature=categorical + categorical_aug)

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'rmse'},
    'subsample': 1,
    'subsample_freq': 1,
    'learning_rate': 0.5,
    'num_leaves': 31,
}

model = lgb.train(
    params,
    d_train,
    num_boost_round=1000,
    valid_sets=(d_train, d_val),
    early_stopping_rounds=100,
    verbose_eval=100,
)

Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.852883	valid_1's rmse: 0.87656
[200]	training's rmse: 0.806052	valid_1's rmse: 0.839098
[300]	training's rmse: 0.775973	valid_1's rmse: 0.824881
[400]	training's rmse: 0.756213	valid_1's rmse: 0.813853
[500]	training's rmse: 0.741446	valid_1's rmse: 0.805921
[600]	training's rmse: 0.730109	valid_1's rmse: 0.798787
[700]	training's rmse: 0.718526	valid_1's rmse: 0.794136
[800]	training's rmse: 0.710251	valid_1's rmse: 0.790784
[900]	training's rmse: 0.703305	valid_1's rmse: 0.787526
[1000]	training's rmse: 0.695985	valid_1's rmse: 0.78456
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 0.695985	valid_1's rmse: 0.78456


In [54]:
# Re-train with fixed iteration, on all data
d_train = lgb.Dataset(
    feature_preproc.transform(df),
    label=df['log_meter_reading'], 
    categorical_feature=categorical + categorical_aug,
)

model = lgb.train(
    params,
    d_train,
    num_boost_round=1000,
)

In [55]:
y_pred = model.predict(feature_preproc.transform(df))

In [56]:
from sklearn.metrics import mean_squared_error

mean_squared_error(df['log_meter_reading'], y_pred)

0.5035213087834565

# Feature importance

In [57]:
pd.DataFrame({
    'feature_name': feature_names,
    'feature_importance': model.feature_importance(),
}).sort_values(by='feature_importance', ascending=False)

,feature_name,feature_importance
9,building_id,9087
14,month,3613
3,air_temperature,3081
7,sea_level_pressure,2896
10,meter,2798
5,dew_temperature,2781
0,square_feet,1221
4,cloud_coverage,1221
12,wind_direction_sin,957
13,wind_direction_cos,794


# Save model

In [58]:
import dill as pickle

model.save_model('model/model_lightgbm.p')
    
with open('model/feature_preproc.p', 'wb') as file:
    pickle.dump(feature_preproc, file)

# Load model

In [59]:
import dill as pickle
import lightgbm as lgb

model = lgb.Booster(model_file='model/model_lightgbm.p')

with open('model/feature_preproc.p', 'rb') as file:
    feature_preproc = pickle.load(file)

# Test

In [61]:
df = test
df = df.merge(building_metadata, on='building_id')

weather_test_full = df[['site_id', 'timestamp']].drop_duplicates()\
    .merge(weather_test, on=['site_id', 'timestamp'], how='left')\
    .groupby(by='site_id', group_keys=False).apply(interpolate).reset_index(drop=True)
    
df = df.merge(weather_test_full, on=['site_id', 'timestamp'], how='left')

In [62]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [63]:
X_test = feature_preproc.transform(df)

In [64]:
X_test.head()

,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_speed,building_id,meter,primary_use,wind_direction_sin,wind_direction_cos,month
0,-0.856955,2.141835,-0.119116,0.175759,0.478239,0.397803,-0.157098,0.776016,0.101319,0.0,0.0,0.0,0.984808,-0.173648,0.0
1,-0.856955,2.141835,-0.119116,0.175759,-0.258084,0.505557,-0.157098,0.862522,-0.119416,0.0,0.0,0.0,0.766044,-0.642788,0.0
2,-0.856955,2.141835,-0.119116,0.021196,-0.994408,0.505557,-0.157098,0.848105,-0.119416,0.0,0.0,0.0,0.642788,-0.766044,0.0
3,-0.856955,2.141835,-0.119116,0.121207,-0.994408,0.554535,-0.157098,0.891358,-0.119416,0.0,0.0,0.0,0.642788,-0.766044,0.0
4,-0.856955,2.141835,-0.119116,0.075747,-0.258084,0.554535,-0.157098,0.905776,-0.340151,0.0,0.0,0.0,0.766044,-0.642788,0.0


In [65]:
y_pred = model.predict(X_test)

In [68]:
result = test[['row_id']].merge(
    pd.DataFrame({'row_id': df['row_id'], 'meter_reading': np.round(np.exp(y_pred) - 1, 4)}), 
    on='row_id', how='left',
)
# Default value for missing rows
result['meter_reading'] = result['meter_reading'].fillna(0.0)

In [67]:
result.to_csv('output/submit.csv', index=False)